# Import paths from environment variables

In [1]:
from dotenv import load_dotenv
import os
import geemap
import ee
import pandas as pd
import time
from geopy import distance

# Load the environment variables from the .env file
load_dotenv()

# os.environ.clear()

# Retrieve paths from environment variables file
path_to_folder = os.getenv('PATH_TO_FOLDER')

path_in = os.path.join(path_to_folder, os.getenv('NIGHTTIME_INPUT'))
path_out = os.path.join(path_to_folder, os.getenv('NIGHTTIME_RAW_OUTPUT'))
path_out_processed = os.path.join(path_to_folder, os.getenv('NIGHTTIME_PROCESSED_OUTPUT'))

if path_in is None and path_out is None and path_out_processed is None:
    print("Paths not found in .env file")
else:
    print("Input path retrieved:", path_in)
    print("Output path retrieved:", path_out)
    print("Output path processed retrieved:", path_out_processed)

local_cell_coordinate_file = path_in + '/local_cell_coordinates.xlsx'
print("Local coordinates file path:", local_cell_coordinate_file)

out_landsat_annual = path_out + '/local_ndvi_landsat_annual.csv'
print("Intermediate output file path:", out_landsat_annual)


Input path retrieved: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/nighttime/ivas
Output path retrieved: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/nighttime/ivas/earth_engine_output
Output path processed retrieved: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/processed_data/nighttime/ivas
Local coordinates file path: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/nighttime/ivas/local_cell_coordinates.xlsx
Intermediate output file path: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/nighttime/ivas/earth_engine_output/local_ndvi_landsat_annual.csv


# Set-up Google Earth Engine

In [2]:
geemap.update_package()
ee.Initialize()

Unzipping geemap-master.zip ...
Data downloaded to: /Users/vaiostriantafyllou/Downloads/geemap-master
Processing /Users/vaiostriantafyllou/Downloads/geemap-master
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for geemap: filename=geemap-0.30.4-py2.py3-none-any.whl size=2196639 sha256=ec64d9765b7dbd8e6ec88db2547cd1d834b03bf2df23e16de0153db028801caf
  Stored in directory: /Users/vaiostriantafyllou/Library/Caches/pip/wheels/97/00/e0/de67a6fb80288e0af3875f1cb330e7e835566c0ae5606b32df
Successfully built geemap
  Attempting uninstall: geemap
    Found existing installation: geemap 0.30.4
    U

# Parameters

In [3]:
# Location of Salar de Atacama
longitude = -68.25180257565476
latitude = -23.498660512303832

# Origin coordinates - Salar de Atacama
origin = str(latitude)+','+str(longitude)

# Surrounding distance for data collection
surrounding_distance = 200000

# Date range for data collection
start_date = '2013-01-01'
end_date = '2021-01-01'

# Functions

In [4]:
# Function for calculating distance using coordinates

def calc_distance(dataf):
    measure_coords = dataf["coordinates"]
    origin_coords = dataf["origin"]
    d = distance.distance(measure_coords, origin_coords).km
    return(d)

# Extract data from Google Earth Engine

In [5]:
# data extraction
selected_point = ee.Geometry.Point([longitude, latitude])
buffer = selected_point.buffer(surrounding_distance)

landsat_collection = ee.ImageCollection("NOAA/VIIRS/DNB/ANNUAL_V21").filterDate(start_date, end_date)\
    .filterBounds(buffer)

annual_imagery = landsat_collection.toBands()

In [10]:
# Define a function to process each row
def process_row(row):
    result = pd.DataFrame()  # Initialize an empty DataFrame to store results

    lat1_col = f'lat_s'
    lat2_col = f'lat_s'
    long1_col = 'long_w'
    long2_col = 'long_e'
    
    data = ee.Geometry.BBox(row[long1_col], row[lat1_col], row[long2_col], row[lat2_col])
    grid_deg = 0.00045  # Set your desired grid size in degrees
    grid_size = 50  # Set your desired pixel scale in meters

    fishnet = geemap.fishnet(data, h_interval=grid_deg, v_interval=grid_deg, delta=1)
    geemap.zonal_statistics(annual_imagery, fishnet, out_landsat_annual, statistics_type='MEAN', scale=grid_size)
    aux = pd.read_csv(out_landsat_annual)
    result = pd.concat([result, aux])

    time.sleep(2)

    return result

variations = ['a', 'b', 'c']  # Include an empty string for the default case

for i in range(1, 21):
    # Determine the file suffix
    file_suffixes = [f'{i}{variation}' for variation in variations] if i in [1, 12] else [str(i)]

    for suffix in file_suffixes:
        local_output = os.path.join(path_out, f'local_{suffix}.csv')
        cell_coordinates = pd.read_excel(local_cell_coordinate_file, sheet_name='summ')

        cell_coordinates['no'] = cell_coordinates['no'].astype(str)

        cell_coordinates = cell_coordinates[cell_coordinates['no'] == str(suffix)]

        # Apply the process_row function to each row of the DataFrame
        result_list = cell_coordinates.apply(process_row, axis=1)

        print(result_list)

        # Concatenate the individual results into a single DataFrame
        final_result = pd.concat(result_list.tolist(), ignore_index=True)

        # Save the final result DataFrame to a CSV file
        final_result.to_csv(local_output)

        print(f"Done with {suffix}")
        time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/nighttime/ivas/earth_engine_output/local_ndvi_landsat_annual.csv
0         20130101_average  20130101_average_masked...
dtype: object
Done with 1a
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/nighttime/ivas/earth_engine_output/local_ndvi_landsat_annual.csv
1         20130101_average  20130101_average_masked...
dtype: object
Done with 1b
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/nighttime/ivas/earth_engine_output/local_ndvi_landsat_annual.csv
2         20130101_average  20130101_average_masked...
dtype: object
Done with 1c
Comp

## Calculate distances from the center of the Salar

In [7]:
# List of file names
file_names = ["local_1a", "local_1b", "local_1c"] + [f"local_{i}" for i in range(2, 12)] + ["local_12a", "local_12b", "local_12c"] + [f"local_{i}" for i in range(13, 21)]

def determine_group(file_name, idx):
    parts = file_name.split('_')  # Adjust the split based on your file name format
    for part in parts:
        if part in ["1a", "1b", "1c"]:
            return 1
        elif part in ["12a", "12b", "12c"]:
            return 12
        elif part in ["2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]:
            return idx - 2
        elif part in ["13", "14", "15", "16", "17", "18", "19", "20"]:
            return idx - 4

# Iterate over each file, process it, and save the processed version
for idx, file_name in enumerate(file_names, start=1):
    # Load each file
    df = pd.read_csv(f"{path_out}/{file_name}.csv")

    # Assign group - here, 'file_name' variable from the loop should be used directly
    df["group"] = determine_group(file_name, idx)

    # Drop unnecessary columns
    df = df.drop(columns=['Unnamed: 0', 'system:index'])

    # Calculate longitude, latitude, and coordinates
    df['longitude'] = (df['east'] + df['west']) / 2
    df['latitude'] = (df['north'] + df['south']) / 2
    df['coordinates'] = df['latitude'].astype(str) + ',' + df['longitude'].astype(str)

    # Assign origin and calculate distance
    df["origin"] = origin
    df['distance'] = df.apply(calc_distance, axis=1)

    # Save the processed file
    df.to_csv(f"{path_out_processed}/{file_name}_with_distances.csv", index=False)


In [8]:
import pandas as pd
import os
import rasterio
import re

# Assuming 'file_names' and 'path_out_processed' are already defined

def file_bounds(hgt_filename):
    # Regex to extract lat and lon from filename
    match = re.match(r'([NS])(\d+)([EW])(\d+).hgt', hgt_filename)
    if match:
        lat_dir, lat, lon_dir, lon = match.groups()
        lat = int(lat) * (1 if lat_dir == 'N' else -1)
        lon = int(lon) * (1 if lon_dir == 'E' else -1)
        return lat, lon
    else:
        raise ValueError(f"Filename format not recognized: {hgt_filename}")


def get_elevation_from_hgt(lat, lon, hgt_files):
    for file in hgt_files:
        file_lat, file_lon = file_bounds(os.path.basename(file))
        if file_lat <= lat < file_lat + 1 and file_lon <= lon < file_lon + 1:
            with rasterio.open(file) as dataset:
                row, col = dataset.index(lon, lat)
                return dataset.read(1)[row, col]
    return None

def get_elevations_for_batch(coords_batch, hgt_files):
    return [get_elevation_from_hgt(lat, lon, hgt_files) for lat, lon in coords_batch]

def process_in_batches(coords, batch_size=50):
    for i in range(0, len(coords), batch_size):
        yield coords[i:i + batch_size]

# List all your HGT files here
hgt_files = ['S24W070.hgt', 'S24W069.hgt', 'S24W068.hgt', 'S23W070.hgt', 'S23W069.hgt', 'S23W068.hgt']  # Replace with paths to your HGT files

for file_name in file_names:
    processed_file_path = f"{path_out_processed}/{file_name}_with_distances.csv"
    processed_file_path_elevation = f"{path_out_processed}/{file_name}_with_distances_and_elevation.csv"
    df = pd.read_csv(processed_file_path)

    # Extract coordinates
    coords = df[['latitude', 'longitude']].values

    # Fetch elevations in batches and update the DataFrame
    updated_elevations = []
    for batch in process_in_batches(coords, batch_size=500):
        batch_elevations = get_elevations_for_batch(batch, hgt_files)
        updated_elevations.extend(batch_elevations)
        print(batch_elevations)

    df['elevation'] = updated_elevations

    # Save the updated file
    df.to_csv(processed_file_path_elevation, index=False)


[2544, 2525, 2560, 2549, 2508, 2535, 2549, 2576, 2541, 2532, 2511, 2536, 2542, 2527, 2473, 2540, 2493, 2439, 2430, 2426, 2419, 2418, 2416, 2415]
[2500, 2478, 2464, 2490, 2473, 2515, 2500, 2413, 2426, 2428, 2417, 2405, 2400, 2396, 2389, 2386, 2386, 2386, 2383, 2385, 2391]
[2475, 2462, 2455, 2433, 2415, 2406, 2415, 2392, 2389, 2389, 2388, 2380, 2369, 2374, 2372, 2373, 2376, 2377, 2376, 2376, 2379, 2381, 2381, 2376]
[3266, 3294, 3369, 3419, 3382, 3309, 3320, 3347, 3363, 3401, 3423, 3433, 3445, 3429, 3479, 3547, 3516]
[2320, 2320, 2321, 2322, 2325, 2323, 2325, 2327, 2327, 2330, 2333, 2332, 2338, 2339, 2345, 2353, 2354, 2364, 2359, 2370]
[2317, 2319, 2318, 2318, 2319, 2316, 2318, 2319, 2319, 2320, 2319]
[2303]
[2302, 2304, 2302, 2302, 2302, 2304, 2304, 2306, 2308, 2307, 2308, 2311, 2316]
[3914]
[3966, 3966, 3976, 3988, 4005, 4037, 4073, 4101, 4137, 4168, 4197, 4207, 4222, 4265, 4356, 4327, 4267, 4264, 4311, 4286, 4251, 4185, 4162, 4147, 4188, 4177, 4203, 4226, 4252, 4270, 4261, 4256, 4317, 

# Calculate distances from individual wells

In [9]:
# Define well locations
wells = {
    "ca2015": (-23.539376, -68.058171),
    "socaire5": (-23.451465, -68.039001),
    "allana1": (-23.373125, -68.031744),
    "camar2": (-23.418412, -68.040096),
    "mullay1": (-23.302575, -68.022884)
}

def calc_distance_from_well(measure_coords, well_coords):
    return distance.distance(measure_coords, well_coords).km

# Process each file
for file_name in file_names:
    df = pd.read_csv(f"{path_out_processed}/{file_name}_with_distances_and_elevation.csv")

    # Add distance calculations for each well
    for well_name, well_coords in wells.items():
        df[f'distance_{well_name}'] = df['coordinates'].apply(lambda x: calc_distance_from_well(x, well_coords))

    # Save the updated file
    df.to_csv(f"{path_out_processed}/{file_name}_with_distances_individual.csv", index=False)